# Chapter 12 - Custom Models and Training with TensorFlow Code Reproduction

In [10]:
# Impor umum
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [11]:
# Membuat Tensor
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix
print("Tensor t:\n", t)

# Indexing
print("\nIndexing t[:, 1:]:\n", t[:, 1:])

# Operasi
print("\nOperasi t + 10:\n", t + 10)
print("Operasi tf.square(t):\n", tf.square(t))
print("Perkalian matriks t @ tf.transpose(t):\n", t @ tf.transpose(t))

# Interoperabilitas dengan NumPy
a = np.array([2., 4., 5.])
print("\nKonversi dari NumPy ke Tensor:", tf.constant(a))
print("Konversi dari Tensor ke NumPy:", t.numpy())

# Variabel TensorFlow
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print("\nVariabel TensorFlow:\n", v)

# Mengubah nilai Variabel
v.assign(2 * v)
print("Variabel setelah v.assign(2 * v):\n", v)
v[0, 1].assign(42)
print("Variabel setelah v[0, 1].assign(42):\n", v)

Tensor t:
 tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)

Indexing t[:, 1:]:
 tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)

Operasi t + 10:
 tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
Operasi tf.square(t):
 tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
Perkalian matriks t @ tf.transpose(t):
 tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)

Konversi dari NumPy ke Tensor: tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)
Konversi dari Tensor ke NumPy: [[1. 2. 3.]
 [4. 5. 6.]]

Variabel TensorFlow:
 <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
Variabel setelah v.assign(2 * v):
 <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
Variabel setelah v[0, 1].assign(42):
 <tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 

In [12]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [13]:
# Fungsi aktivasi kustom (softplus)
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

# Inisialisasi kernel kustom
def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

# Regularizer L1 kustom
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

# Penggunaan komponen kustom dalam sebuah layer
# layer = keras.layers.Dense(1, activation=my_softplus,
#                            kernel_initializer=my_glorot_initializer,
#                            kernel_regularizer=my_l1_regularizer)

In [14]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        # Membuat bobot (weights)
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_uniform")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        # Logika forward pass
        return self.activation(X @ self.kernel + self.bias)

In [15]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z # Koneksi residual

class ResidualRegressor(keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3): # Loop untuk membuat model lebih dalam
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

In [16]:
# Menghitung gradien dari f(x) = x^2
w = tf.Variable(5.0)
with tf.GradientTape() as tape:
    z = w ** 2

dz_dw = tape.gradient(z, w)
print("\nGradien z terhadap w:", dz_dw)


Gradien z terhadap w: tf.Tensor(10.0, shape=(), dtype=float32)


In [17]:
# Menyiapkan data dan model sederhana
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

# Menyiapkan komponen pelatihan
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.MeanSquaredError()

# Loop pelatihan kustom
print("\nMemulai Loop Pelatihan Kustom:")
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        # Mengambil satu batch data
        X_batch, y_batch = X_train[(step-1)*batch_size:step*batch_size], y_train[(step-1)*batch_size:step*batch_size]

        # Membuka GradientTape
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses) # Menambahkan loss regularisasi

        # Menghitung dan menerapkan gradien
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))


Memulai Loop Pelatihan Kustom:
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [18]:
# Fungsi Python biasa
def cube(x):
    return x ** 3
print("\nHasil cube(2):", cube(2))

# Mengubahnya menjadi TensorFlow Function
tf_cube = tf.function(cube)
print("Hasil tf_cube(2):", tf_cube(2))

# Menggunakan decorator (cara yang lebih umum)
@tf.function
def tf_cube_decorated(x):
    return x ** 3

print("Hasil tf_cube_decorated(2):", tf_cube_decorated(2))

# Melihat kode yang dihasilkan oleh AutoGraph
print("\nKode yang dihasilkan AutoGraph:")
print(tf.autograph.to_code(tf_cube_decorated.python_function))


Hasil cube(2): 8
Hasil tf_cube(2): tf.Tensor(8, shape=(), dtype=int32)
Hasil tf_cube_decorated(2): tf.Tensor(8, shape=(), dtype=int32)

Kode yang dihasilkan AutoGraph:
def tf__tf_cube_decorated(x):
    with ag__.FunctionScope('tf_cube_decorated', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = ag__.ld(x) ** 3
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

